# Creating the action server

In this section, we'll discuss **demo_action_server.py**. The action server receives a goal value that is a number. When the server gets this goal value, it'll start counting from zero to this number. If the counting is complete, it'll successfully finish the action, if it is preempted before finishing, the action server will look for another goal value.

To do this, first create a python file called **demo_action_server.py** in the scripts folder of action_tutorials package. Then, copy the following into it:

In [ ]:
#! /usr/bin/env python

import actionlib
import rospy

from action_tutorials.msg import DemoFeedback, DemoResult, DemoAction

class DemoClass(object):

	_feedback = DemoFeedback()
	_result   = DemoResult()

	_feedback.current_number = 0

	def __init__(self):
		self._as = actionlib.SimpleActionServer("demo_as", DemoAction, self.goal_callback, False)
		self._as.start()

	def goal_callback(self, goal):
		r = rospy.Rate(1)
		success = True

		progress = goal.count
		for i in range(0, progress):

			if self._as.is_preempt_requested():
				rospy.loginfo("The goal has been cancelled")
				self._as.set_preempted()
				success = False
				break

			self._feedback.current_number += 1
			self._as.publish_feedback(self._feedback)

			r.sleep()

		if success:
			self._result.final_count = self._feedback.current_number
			rospy.loginfo("Succeeded! Final goal, count = %s" % self._result.final_count)
			self._as.set_succeeded(self._result)

if __name__ == '__main__':
	rospy.init_node("demo")
	DemoClass()
	rospy.spin()

Compile and run it.

In [ ]:
roscore

In [ ]:
rosrun action_tutorials demo_action_server.py

You won't see anything. Nothing happens right now. Let's check our nodes and topics list in the new tab:

In [ ]:
rosnode list

#### /demo
#### /rosout

In [ ]:
rostopic list

** /demo_as/cancel **
<t>
** /demo_as/feedback **
<t>
**/demo_as/goal**
<t>
**/demo_as/result**
<t>
**/demo_as/status**
<t>
**/rosout**
<t>
**/rosout_agg**

You'll see the **'demo_as'** action server topics.

Okay, in the new tab, let's echo the feedback topic.

In [ ]:
rostopic echo /demo_as/feedback

Nothing happens rightnow. Because we don't give any goals to the server. So, let's try now by publishing:

In [ ]:
rostopic pub /demo_as/goal action_tutorials/DemoActionGoal "header: 
  seq: 0
  stamp:
    secs: 0
    nsecs: 0
  frame_id: ''
goal_id:
  stamp:
    secs: 0
    nsecs: 0
  id: ''
goal:
  count: 10" 

In the feedback tab, you will see the count messages increasing from 1 to the goal. For us, the goal is 10.
<t>
When the count reaches 10, you'll see the success message in the server tab.

#### [INFO] [1519666564.791267]: Succeeded! Final goal, count = 10


You can also cancel the process by publishing through the cancel topic to the server. Let's do it.

First, close the goal publisher teminal and publish again the goal with more counts.

In [ ]:
rostopic pub /demo_as/goal action_tutorials/DemoActionGoal "header: 
  seq: 0
  stamp:
    secs: 0
    nsecs: 0
  frame_id: ''
goal_id:
  stamp:
    secs: 0
    nsecs: 0
  id: ''
goal:
  count: 50" 

While running, open new tab and publish cancel request to the server

In [ ]:
rostopic pub /demo_as/cancel actionlib_msgs/GoalID "stamp:
  secs: 0
  nsecs: 0
id: ''" 

In [2]:
range(0, 10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

This time you'll see the cancel message in the server terminal:

#### [INFO] [1519666873.781875]: The goal has been cancelled


### Yes, that's all. You can now create your own action server.
<t>
### In the next section, we'll create our own action client, see you!